<a href="https://colab.research.google.com/github/germancruzram/EXCEL-TO-KML/blob/main/XLS_a_KML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **# ESTE CUADERNO EN COLAB PERMITE CREAR UN KML PARTIENDO DE UNA TABLA EXCEL CON COORDENADAS GEOGRAFICAS CON GRADOS DECIMALES.**

**Elaborado por: German Ahmed Cruz Ramírez. 🇳**

**1. Preparemos el ambiente de trabajo, cargando las librerias python a utilizar**

In [ ]:
!pip install simplekml
!pip install folium
!pip install gdal
import pandas as pd
import simplekml
import os
import json
from osgeo import ogr


**2. CREAREMOS LAS CARPETAS PARA SUBIR LA TABLA EXCEL CON COORDENADAS Y LA CARPETA DE SALIDA KML**

In [30]:
import os

# Define el nombre de las carpetas
carpeta_xls = "XLS"
carpeta_kml = "KML"

# Crea la carpeta "XLS" si no existe
if not os.path.exists(carpeta_xls):
    os.makedirs(carpeta_xls)
    print(f'Creada la carpeta "{carpeta_xls}, ahora puedes subir la tabla con coordenadas"')
else:
    print(f'La carpeta "{carpeta_xls}" ya existe y puedes subir la tabla con coordenadas')

# Crea la carpeta "KML" si no existe
if not os.path.exists(carpeta_kml):
    os.makedirs(carpeta_kml)
    print(f'Creada la carpeta "{carpeta_kml}"')
else:
    print(f'La carpeta "{carpeta_kml}" ya existe')


La carpeta "XLS" ya existe y puedes subir la tabla con coordenadas
La carpeta "KML" ya existe


**💡 RECUERDA QUE LA TABLA EXCEL DEBE CONTENER UNA COLUMNA "LATITUD" Y "LONGITUD" EN GRADOS DECIMALES**

**3. PROCESANDO LA INFORMACION PARA CREAR EL KML**

In [37]:
import pandas as pd
import simplekml
import os

# Define las carpetas con rutas absolutas
carpeta_xls = "/content/XLS"
carpeta_kml = "/content/KML"

# Solicitar al usuario el nombre del campo para el nombre de los puntos
nombre_campo = input("Por favor, ingresa el nombre del campo para el nombre de los puntos en el KML: ")

# Listar los archivos en la carpeta XLS y filtrar por archivos Excel
archivos_xls = [archivo for archivo in os.listdir(carpeta_xls) if archivo.endswith('.xlsx') or archivo.endswith('.xls')]

# Procesar cada archivo Excel encontrado
for nombre_archivo in archivos_xls:
    ruta_excel = os.path.join(carpeta_xls, nombre_archivo)
    df = pd.read_excel(ruta_excel)

    # Crea el objeto KML
    kml = simplekml.Kml()

    # Itera sobre cada fila del DataFrame y añade un punto en el KML
    for idx, fila in df.iterrows():
        descripcion = '<br>'.join([f'{col}: {fila[col]}' for col in df.columns if col not in ['Latitud', 'Longitud']])
        nombre_punto = fila.get(nombre_campo, "Sin información")
        punto = kml.newpoint(name=nombre_punto, coords=[(fila['Longitud'], fila['Latitud'])])
        punto.description = descripcion

    # Guarda el archivo KML en la carpeta "KML"
    nombre_kml = nombre_archivo.replace('.xlsx', '.kml').replace('.xls', '.kml')
    ruta_kml = os.path.join(carpeta_kml, nombre_kml)
    kml.save(ruta_kml)
    print(f"Archivo KML guardado en: {ruta_kml}")


Por favor, ingresa el nombre del campo para el nombre de los puntos en el KML: PK_ubica
Archivo KML guardado en: /content/KML/signal.kml


In [38]:
# Convertir KML a GeoJSON
def kml_to_geojson(kml_path, geojson_path):
    driver = ogr.GetDriverByName('KML')
    data = driver.Open(kml_path)
    if data is None:
        raise Exception(f"Error al abrir el archivo KML: {kml_path}")
    layer = data.GetLayer()
    driver = ogr.GetDriverByName('GeoJSON')
    if os.path.exists(geojson_path):
        driver.DeleteDataSource(geojson_path)
    outData = driver.CreateDataSource(geojson_path)
    outLayer = outData.CreateLayer('layer', geom_type=ogr.wkbMultiPolygon)
    layerDefn = layer.GetLayerDefn()
    for i in range(layerDefn.GetFieldCount()):
        fieldDefn = layerDefn.GetFieldDefn(i)
        outLayer.CreateField(fieldDefn)
    outLayerDefn = outLayer.GetLayerDefn()
    for feature in layer:
        geom = feature.GetGeometryRef()
        outFeature = ogr.Feature(outLayerDefn)
        outFeature.SetGeometry(geom)
        for i in range(0, outLayerDefn.GetFieldCount()):
            outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(), feature.GetField(i))
        outLayer.CreateFeature(outFeature)
        outFeature = None
    outData = None

# Definición de rutas
carpeta_kml = "/content/KML"
nombre_kml = "signal.kml"  # Cambia esto por el nombre de tu archivo KML
nombre_geojson = "signal.geojson"
ruta_kml = os.path.join(carpeta_kml, nombre_kml)
ruta_geojson = os.path.join(carpeta_kml, nombre_geojson)

# Convertir KML a GeoJSON
kml_to_geojson(ruta_kml, ruta_geojson)

# Cargar el GeoJSON en un mapa de Folium
m = folium.Map(location=[0, 0], zoom_start=2)

# Función para añadir popups que muestran toda la información disponible
def add_popups(feature):
    properties = feature['properties']
    html = '<br>'.join([f'{key}: {value}' for key, value in properties.items()])
    return folium.Popup(html)

# Añadir GeoJSON al mapa con popups y tooltips
folium.GeoJson(
    ruta_geojson,
    name="GeoJSON data",
    style_function=lambda feature: {
        'fillColor': '#ffff00',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['Name', 'Description'],  # Ajusta estos campos según los datos disponibles en tu GeoJSON
        aliases=['Nombre:', 'Descripción:']
    ),
    popup=add_popups
).add_to(m)

folium.LayerControl().add_to(m)

# Mostrar el mapa
m

